In [1]:
import pandas as pd
from engine.hyperparameters_tune_raytune import deepscreen_hyperparameter_tuneing
from utils.configurations import configs
import os

/home/sjinich/disco/che_env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-02-23 17:32:20,278	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-02-23 17:32:20,441	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-02-23 17:32:22,194	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [2]:
import warnings
warnings.filterwarnings("ignore", ".*Consider increasing the value of the `num_workers` argumen*")

data = pd.read_csv("/home/sjinich/disco/TrypanoDEEPscreen/.data/processed/CHEMBL2581.csv")
    
search_space_deepscreen = configs.get_hyperparameters_search()

tuner = deepscreen_hyperparameter_tuneing(
    data=data,
    data_split_mode="non_random_split",
    search_space=search_space_deepscreen,
    target="chembl2581_trial",
    experiments_result_path="../../.experiments",
    **configs.get_hyperparameters_search_setup()
    )

result = tuner.tune_deepscreen()


(TorchTrainer pid=1319480) Started distributed worker processes: 
(TorchTrainer pid=1319480) - (ip=10.1.103.91, pid=1320314) world_rank=0, local_rank=0, node_rank=0
(RayTrainWorker pid=1320262) Setting up process group for: env:// [rank=0, world_size=1]
(RayTrainWorker pid=1320262) INFO: Using hyperparameters [('fully_layer_1', 128), ('fully_layer_2', 512), ('drop_rate', 0.3), ('learning_rate', 0.005), ('batch_size', 32), ('experiment_result_path', '/big/lab/sjinich/TrypanoDEEPscreen/.experiments/chembl2581_trial')]
(RayTrainWorker pid=1320262) GPU available: False, used: False
(RayTrainWorker pid=1320262) TPU available: False, using: 0 TPU cores
(RayTrainWorker pid=1320262) IPU available: False, using: 0 IPUs
(RayTrainWorker pid=1320262) HPU available: False, using: 0 HPUs
(RayTrainWorker pid=1320262) [rank: 0] Seed set to 123
(RayTrainWorker pid=1320262) Missing logger folder: /home/sjinich/ray_results/TorchTrainer_2024-02-23_17-32-22/TorchTrainer_823bd_00000_0_batch_size=32,drop_rat

In [10]:
ckpt_dir = os.path.join(result.get_best_result(metric="val_mcc",mode="max").checkpoint.path,"checkpoint.ckpt")

Result(
  metrics={'train_loss': 0.4247746169567108, 'val_loss': 0.4044719636440277, 'val_acc': 0.8324176073074341, 'val_f1': 0.8688172101974487, 'val_mcc': 0.6386048197746277, 'val_prec': 0.848739504814148, 'val_recall': 0.8898678421974182, 'train_acc': 0.8836050629615784, 'train_f1': 0.8851717114448547, 'train_mcc': 0.7681795358657837, 'train_prec': 0.868130624294281, 'train_recall': 0.9106861352920532, 'epoch': 9, 'step': 460},
  path='/home/sjinich/ray_results/TorchTrainer_2024-02-23_17-32-22/TorchTrainer_823bd_00001_1_batch_size=32,drop_rate=0.3000,fully_layer_1=256,fully_layer_2=32,learning_rate=0.0001_2024-02-23_17-32-25',
  filesystem='local',
  checkpoint=Checkpoint(filesystem=local, path=/home/sjinich/ray_results/TorchTrainer_2024-02-23_17-32-22/TorchTrainer_823bd_00001_1_batch_size=32,drop_rate=0.3000,fully_layer_1=256,fully_layer_2=32,learning_rate=0.0001_2024-02-23_17-32-25/checkpoint_000009)
)

In [9]:
from engine.system import DEEPScreenClassifier
from datasets.datamodule import DEEPscreenDataModule

model = DEEPScreenClassifier.load_from_checkpoint(result.get_best_result(metric="val_mcc",mode="max").checkpoint.path,experiment_result_path="../../.experiments/chembl2581")

IsADirectoryError: [Errno 21] Is a directory: '/home/sjinich/ray_results/TorchTrainer_2024-02-23_17-32-22/TorchTrainer_823bd_00001_1_batch_size=32,drop_rate=0.3000,fully_layer_1=256,fully_layer_2=32,learning_rate=0.0001_2024-02-23_17-32-25/checkpoint_000009'